# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset using brute force search. 

First we import the necessary packages and create the global variables.

In [3]:
import math
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

# Define the model

In [4]:
def RULmodel_SN(input_shape):
    
    lambda_regularization = 0.20
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Define objective function

Define the function that evaluates each set of data-related params and returns the RMSE as value.

In [5]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    #windows_size = maxWindowSize[datasetNumber]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel("RUL_SN_optmizable", modelRULSN, selected_features, '../CMAPSSData', 'keras',
                          scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData(rectify_labels = False)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.trainTime
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateModel(["rhs"], round=2)
    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

# Optimize the parameters for the NN using DE

In [ ]:
#Optimize the parameters for the NN using DE

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
maxWindowSize = {'1':30, '2':20} #Do it only for datasets 1 and 2
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

# for datasetNumber in maxWindowSize:
    
#    print("Tunning for dataset "+datasetNumber)
#     file = open("intermediateResults6_notRectified"+datasetNumber+".txt", "w")

#     windowSizeBounds = [1,maxWindowSize[datasetNumber]]
#     windowStrideBounds = [1,10]
#     constantRULBounds = [90,140]

#     #bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]
#     bounds = [windowStrideBounds, constantRULBounds] #Optimize only 2 variabes


#     startTime = time.clock()
#     tempResults = differential_evolution(nn_optmize_fun, bounds, 
#                                      args=(selected_features, datasetNumber, min_max_scaler, 0, 20, file, 1),
#                                     strategy='best1bin', maxiter=20, popsize=30, disp=True)
#     results[datasetNumber] = tempResults
#     endTime = time.clock()

#     file.close()
#     totalTime[datasetNumber] = endTime - startTime

for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("intermediateResultsbruteforce_notRectified"+datasetNumber+".txt", "w")
    startTime = time.clock()
    for w in range(15,maxWindowSize[datasetNumber]+1):
        
        for s in range(1,11):
            for r in range(90,141):
                
                x = np.array([w,s,r])
                
                results = nn_optmize_fun(x,selected_features=selected_features,datasetNumber=datasetNumber, 
                                         scaler = min_max_scaler, verbose=0, epochs=20, 
                                        saveToFile = file, iterations = 0)
    endTime = time.clock()
    file.close()
    totalTime[datasetNumber] = endTime - startTime


Tunning for dataset 1
Creating model
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Training model
Training time 2.6263802038634907
Assesing model performance
100/100 [==============================] - 0s 170us/step
Creating model
Training model
Training time 2.446787667545692
Assesing model performance
100/100 [==============================] - 0s 169us/step
Creating model
Training model
Training time 2.6850021747780346
Assesing model performance
100/100 [==============================] - 0s 170us/step
Creating model
Training model
Training time 2.7104741406213435
Assesing model performance
100/100 [==============================] - 0s 209us/step
Creating model
Training model
Training time 2.5020477534445433
Assesing model performance
100/100 [==============================] - 0s 170us/step
Creating model
Training model
Training time 2.5166190100956527
Assesing model performance
100/100 

Training time 1.3379968343813857
Assesing model performance
100/100 [==============================] - 0s 170us/step
Creating model
Training model
Training time 1.3803557566277505
Assesing model performance
100/100 [==============================] - 0s 170us/step
Creating model
Training model
Training time 1.4139590029959095
Assesing model performance
100/100 [==============================] - 0s 169us/step
Creating model
Training model


In [9]:
print("Total time {}".format(totalTime))
print(results)

Total time {'4': 0, '1': 18118.907471727885, '2': 15901.300738888152, '3': 0}
59.27322450422804
